# Deutsch's Algorithm

In [1]:
try:
    import cirq as cirq
except ImportError:
    print("installing cirq...")
    !pip3 install --quiet cirq
    print("installed cirq.")

In [53]:
def deutschs(functionImage):
    circuit = cirq.Circuit()
    q0, q1 = cirq.LineQubit.range(2)
    circuit.append([cirq.X(q1), cirq.H(q1), cirq.H(q0)])
    if functionImage[0]:
        circuit.append([cirq.CNOT(q0, q1), cirq.X(q1)])
    if functionImage[1]:
        circuit.append(cirq.CNOT(q0, q1))
    circuit.append([cirq.H(q0), cirq.measure(q0, key='result')])
    print(circuit)
    simulator = cirq.Simulator()
    result = simulator.run(circuit)
    return not result.measurements['result']

In [58]:
image = [1,0]
if deutschs(image):
    print('func is constant')
else:
    print('func is not constant')

0: ───H───────@───H───M('result')───
              │
1: ───X───H───X───X─────────────────
func is not constant


In [5]:
import cirq as cirq
mycircuit = cirq.Circuit()
q0, q1 = cirq.LineQubit.range(2)
def gen():
    yield [cirq.CNOT(q0, q1), cirq.X(q1)]
    yield cirq.CNOT(q0, q1)

g = gen()
mycircuit.append(g)
print(mycircuit)


0: ───@───────@───
      │       │
1: ───X───X───X───


## Two bit input

In [3]:
q0, q1, q2 = cirq.LineQubit.range(3)

constant = (
    [], 
    [cirq.X(q2)]
)

balanced = (
    [cirq.CNOT(q0, q2)], 
    [cirq.CNOT(q1, q2)], 
    [cirq.CNOT(q0, q2), cirq.CNOT(q1, q2)],
    [cirq.CNOT(q0, q2), cirq.X(q2)], 
    [cirq.CNOT(q1, q2), cirq.X(q2)], 
    [cirq.CNOT(q0, q2), cirq.CNOT(q1, q2), cirq.X(q2)]
)


In [4]:
def dj_circuit(oracle):

    yield cirq.X(q2), cirq.H(q2)
    
    yield cirq.H(q0), cirq.H(q1)
    
    yield oracle
    
    yield cirq.H(q0), cirq.H(q1), cirq.H(q2)
    
    yield cirq.X(q0), cirq.X(q1), cirq.CCX(q0, q1, q2)
    yield cirq.measure(q2)

In [10]:
circuit = cirq.Circuit(dj_circuit([]))
print(circuit)

0: ───H───H───X───@───────
                  │
1: ───H───H───X───@───────
                  │
2: ───X───H───H───X───M───


In [7]:
s = cirq.Simulator()
print("constant functions:")
for oracle in constant:
    result = s.run(cirq.Circuit(dj_circuit(oracle)), repetitions=10)
    print(result)
    
print("balanced functions:")
for oracle in balanced:
    result = s.run(cirq.Circuit(dj_circuit(oracle)), repetitions=10)
    print(result)

constant functions:
2=0000000000
2=0000000000
balanced functions:
2=1111111111
2=1111111111
2=1111111111
2=1111111111
2=1111111111
2=1111111111
